In [3]:
import re#Busqueda de patrones y analisis de texto
import sketch#Dibujos hechos a mano
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import geojson
import folium
from folium.plugins import HeatMap
from summarytools import dfSummary

## ETL Extracción, transformación y carga

In [14]:
#Extraemos y leemos la informacion de los dataset contemplando cuales son las hojas de calulo que necesitamos 
homicidiosHechos = pd.read_excel("homicidios.xlsx", sheet_name="HECHOS")
homicidiosVictimas = pd.read_excel("homicidios.xlsx", sheet_name="VICTIMAS")
comunas= pd.read_excel("comunas.xlsx")

## Tratamiento de los datos del dataset `homicidios - hechos`

In [15]:
homicidiosHechos

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,...,NaN,MOM,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,NaN,LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,...,NaN,TERRADA,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,...,4071.0,NaN,"PERON, EVA AV. 4071",9,Point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS


In [16]:
#Verificamos si existen datos nulos o duplicados 
homicidiosHechos.isnull().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

In [17]:
homicidiosHechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [18]:
homicidiosHechos[homicidios_hechos["Dirección Normalizada"].isnull()][["Dirección Normalizada", "Cruce","Altura", "Calle"]]

,Dirección Normalizada,Cruce,Altura,Calle
38,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
106,NaN,NaN,NaN,AUTOPISTA BUENOS AIRES - LA PLATA
119,NaN,NaN,NaN,NaN
180,NaN,NaN,NaN,AUTOPISTA PERITO MORENO
181,NaN,NaN,NaN,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.
313,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
546,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
621,NaN,NaN,NaN,AUTOPISTA BUENOS AIRES - LA PLATA


In [19]:
homicidiosHechos[["Altura", "Calle", "Dirección Normalizada", "Cruce"]][:5]

,Altura,Calle,Dirección Normalizada,Cruce
0,NaN,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...","FERNANDEZ DE LA CRUZ, F., GRAL. AV."
1,NaN,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",DE LOS CORRALES AV.
2,2034.0,ENTRE RIOS AV.,ENTRE RIOS AV. 2034,NaN
3,NaN,LARRAZABAL AV.,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.","VILLEGAS, CONRADO, GRAL."
4,NaN,SAN JUAN AV.,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.","SAENZ PE?A, LUIS, PRES."


Nuestro DataFrame consta de 696 filas con 21 columnas cada una, esto para un total de 14,616 datos, de estos, 747 son datos nulos pertenecientes a las columnas: **`Altura`**, **`Dirección Normalizada`**, **`Cruce`** y **`Calle`**. 
- Para el caso de **`Calle`** tiene un solo dato nulo.
- Para el caso de  **`Dirección Normalizada`** 8 nulos. 
- Para el caso de **`Cruce`** tiene 171 nulos, siendo el 24.56% del los datos de la columna.
- Para el caso de **`Altura`** tiene 567 nulos que representan el 81.5% de datos de la columna

La razon por la que la columna `Altura` carece de tantos datos es debido a que esta hace referencia al punto en que en una avenida o calle sucede un siniestro y dado que una gran cantidad de los mismos sucede en cruces o intersecciones es muy comun que no se tenga la infomación disponible.

La columna `Cruce` solo va a tener dato cuando el hecho se porduce en un cruce de calles, por lo que se conserva de esta manera.
La columna `Dirección Normalizada` no tiene posibilidad de completarse con las columnas del dataset; pero contiene solo 1,1% de nulos por lo se conserva de esta manera.
 

In [20]:
# Se elimina la columna "Altura"
homicidiosHechos= homicidiosHechos.drop("Altura", axis=1)
homicidiosHechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

In [21]:
homicidiosHechos.duplicated().sum()

0

### Transformación de datos

Con el proposito de mejorar la versatilidad de los datos y su mosterior manipulación la columna `HH`pasara de ser tipo object a int, esto con el proposito de permitir su modificación posterior.


In [22]:
#Eliminando datos duplicados

homicidiosHechos["HH"].unique()

array([4, 1, 7, 0, 5, 18, 19, 15, 11, 22, 16, 9, 23, 6, 10, 17, 12, 8, 20,
       21, 14, 3, 2, 13, 'SD'], dtype=object)

El dato SD por ahora lo mantendremos asi dado que en caso de ser eliminado podria ser perjudicial para nuestra muestra de datos SD, no contemplamos la idea de promediar el resto de datos y suplirlo ya que podria no ser representativo.

In [23]:
# Con el proposito de poder manipular la columna HH se hacen los cambios necesarios para que sea de tipo Int64 
homicidiosHechos["HH"] = pd.to_numeric(homicidiosHechos ["HH"], errors="coerce")
homicidiosHechos["HH"] = homicidiosHechos["HH"].astype("Int64")

In [24]:
# Filtrar filas con posiciones no válidas
valid_positions = ~(homicidiosHechos['pos x'].str.contains('[^0-9.,-]') | homicidiosHechos['pos y'].str.contains('[^0-9.,-]') | (homicidiosHechos['pos x'] == '.') | (homicidiosHechos['pos y'] == '.'))

# Establecer posiciones no válidas en NaN
homicidiosHechos.loc[~valid_positions, ['pos x', 'pos y']] = np.nan

# Convertir las cadenas a tipo numérico (float)
homicidiosHechos['pos x'] = homicidiosHechos['pos x'].str.replace(',', '.').astype(float)
homicidiosHechos['pos y'] = homicidiosHechos['pos y'].str.replace(',', '.').astype(float)

In [25]:
homicidiosHechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO


In [26]:
#La columna `XY (CABA)` posee coordenadas para poder ubicar en un mapa , las extraemos para poder utilizar


def extraer_coordenadas(texto):
    # Utilizamos una expresión regular para extraer las coordenadas
    coordenadas = re.findall(r'\d+\.\d+', texto)
    if len(coordenadas) == 2:
        return float(coordenadas[0]), float(coordenadas[1])
    else:
        return None, None

# Aplicar la función a la columna 'XY (CABA)'
homicidiosHechos ['coordenada_x'], homicidiosHechos['coordenada_y'] = zip(*homicidiosHechos['XY (CABA)'].map(extraer_coordenadas))

# Filtrar los registros que no tienen coordenadas válidas
homicidios_hechos = homicidiosHechos[(homicidiosHechos['coordenada_x'].notnull()) & (homicidiosHechos['coordenada_y'].notnull())]



In [28]:
#verficamos los cambios realizados:

homicidiosHechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO,coordenada_x,coordenada_y
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,106684.290900,99706.576878
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,MOTO,SD,99840.652248,94269.165344
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,106980.328279,100752.169158


In [30]:
#Observemos lo cambios en las columnas 
homicidiosHechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO', 'coordenada_x', 'coordenada_y'],
      dtype='object')

A modo de una buena practica vamos a estandarizar ls mayusculas y minusculas de las columnas:

In [31]:
#Cambio la primer letra a mayúscula
homicidiosHechos.columns = [x.capitalize() for x in homicidiosHechos.columns]
# Reemplazo guiones por espacios
homicidiosHechos.columns = homicidiosHechos.columns.str.replace("_", " ")
#Renombra columnas 
homicidiosHechos= homicidiosHechos.rename(columns={"N victimas": "Num víctimas",
                                            "Aaaa": "Año",
                                            "Mm":"Mes",
                                            "Dd": "Día",
                                            "Hora": "Hora completa",
                                            "Hh": "Hora"}) 

homicidiosHechos.columns

Index(['Id', 'Num víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora completa',
       'Hora', 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Pos x', 'Pos y',
       'Participantes', 'Victima', 'Acusado', 'Coordenada x', 'Coordenada y'],
      dtype='object')

Voy a realizar un analisis de la columna `Día` esto debido a que si lo analizamos en funcion de ello podriamos establecer mediante los `Días semanas` si existe un incremento o decremento en función de si es o no fin de semana.

In [33]:
#Consulto la columna Día
homicidiosHechos.Día

0       1
1       2
2       3
3      10
4      21
       ..
691    13
692    20
693    30
694    15
695    18
Name: Día, Length: 696, dtype: int64

In [35]:
#Creo una columna nueva `Día semana` que contenga el nombre del día de la semana
homicidiosHechos['Día semana'] = homicidiosHechos['Fecha'].dt.strftime('%A')

#Creo un diccionario para modificar a español los nombres de los dás de la semana y lo aplico a la columna nueva
dic={  'Friday':'Viernes',
       'Saturday': 'Sabado',
       'Sunday': 'Domingo',  
       'Thursday': 'Jueves',
        'Monday': 'Lunes',
        'Wednesday': 'Miercoles',
       'Tuesday': 'Martes'}
homicidiosHechos['Día semana'] =homicidiosHechos['Día semana'].replace(dic)

homicidiosHechos.head()

,Id,Num víctimas,Fecha,Año,Mes,Día,Hora completa,Hora,Lugar del hecho,Tipo de calle,...,Comuna,Xy (caba),Pos x,Pos y,Participantes,Victima,Acusado,Coordenada x,Coordenada y,Día semana
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378,Viernes
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420,Sabado
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,106684.290900,99706.576878,Domingo
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,MOTO,SD,99840.652248,94269.165344,Domingo
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,106980.328279,100752.169158,Jueves


In [37]:
homicidiosHechos["Comuna"].unique()

array([ 8,  9,  1, 11, 15,  4,  7, 12,  3, 13, 14, 10,  6,  2,  5,  0],
      dtype=int64)

In [39]:
#Agrego la palabra Comuna delante del dato numérico
#se aplica la función lambda a cada fila del dataframe 
homicidiosHechos["Comuna"] = homicidiosHechos.apply(lambda x: "Comuna "+str(x["Comuna"]), axis=1)

homicidiosHechos["Comuna"].unique()

array(['Comuna Comuna 8', 'Comuna Comuna 9', 'Comuna Comuna 1',
       'Comuna Comuna 11', 'Comuna Comuna 15', 'Comuna Comuna 4',
       'Comuna Comuna 7', 'Comuna Comuna 12', 'Comuna Comuna 3',
       'Comuna Comuna 13', 'Comuna Comuna 14', 'Comuna Comuna 10',
       'Comuna Comuna 6', 'Comuna Comuna 2', 'Comuna Comuna 5',
       'Comuna Comuna 0'], dtype=object)

Se realiza un resumen descriptivo de los datos en `homicidios-hechos`

In [40]:
dfSummary(homicidiosHechos)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Id[object],1. 2016-00012. 2019-00503. 2019-00424. 2019-00435. 2019-00446. 2019-00457. 2019-00468. 2019-00479. 2019-004810. 2019-004911. other,1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)686 (98.6%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADXElEQVR4nO3cwWkbYRRG0TdGOPZGQhhcQtKDi0ixKUJFpAdjPIvY8WaySBYpINwB5ZwK/sWFh2A+Ldu2DRRu9n4A/w+xkREbGbGRERsZsZERGxmxkREbGbGRuVmW5cuyLHd7P4TrdzMzn2fm094P4fo5o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRG5mZmvs/Mz70fwvVb/IsRFWeUjNjIWFeRsa4i44ySERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZExeCFj8ELGGSUjNjIGL2QMXsg4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHFGyYiNjNjIWFeRsa4i44ySERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZGxriJjXUXGGSUjNjIGL2QMXsg4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZGxriJjXUXGGSUjNjLWVWSsq8g4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiMjNjLWVWSsq8g4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryFhXkXFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyy8zcz8zt3g/hqn1s2/Z2eHh4+Ho8Hs97v4brta7ry7Is3w7H4/H89PT0djqd3vd+FNfn9fX17nK5nJ+fn28PMzOn0+n98fHxx94P42rdz/iBQEhsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERuYw8/szkL0fwnX6u63Duq4vl8vlPH8+A4F/bV3Xl5n58Fk4hY9t296sq8j4gUBGbGTERkZsZMRGRmxkxEZGbGTERuYXwgdpOf758ysAAAAASUVORK5CYII="">",0(0.0%)
2,Num víctimas[int64],Mean (sd) : 1.0 (0.2)min < med < max:1.0 < 1.0 < 3.0IQR (CV) : 0.0 (5.7),3 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABs0lEQVR4nO3XQWrCUBCH8ZkiqQ/khZCaY3gAD+FhPYRbTxLkZdGoVF53hVJaTbToX77fNplhFh8RPedswKN7ufcBwCUm515w92Bmxcj9x5xzP3IW+PJnqO4e6rpexRirMcu7rtu5+5pYca1zX9Qixlgtl8u+LMv9kMUppelms6nati3MjFBxlbM//WZmZVnum6Z5H7E/jJgBfuDPFCQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiRM7n3AM3L3YGbFyPFjzrm/5T3PgFBvzN1DXderGGM1Zr7rup27r4n1u4tCTSlNhy5OKU1Pp9PEzKK7Dz5MWJzNZvPFYvERQjgMGez7/nW73c7btn1z9+6/DnxQh5zz/reHnwblSFGwSZ17AAAAAElFTkSuQmCC"">",0(0.0%)
3,Fecha[datetime64[ns]],"Min: 2016-01-01Max: 2021-12-30Duration: 2,190 days",598 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABNCAYAAACxBha+AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACj0lEQVR4nO3dTYriQBiH8bfGVqJCRATdeYGBWbj0EHPYOcIsPIE3cCeIREx3MMbMZmAWPX6k2vzNx/Pbl2TxkHpjCuLyPDdA4durLwDtQWyQeSvjR51zfTPreSw95Xn+8ezrQTU8PTbnXH8ymfwMw3BcdO3hcNg7534RXDOVcWfrhWE4Xi6XH6PRKHl0URRFwWq1Gu92u56ZEVsDlbKNmpmNRqNkOp2+F1zWL+ViUAmlxVYHX5gtzZgvC2ttbF+ZLc2YL320NjbznC3NmC99tTk2M/OeLc2YLwvjT13INOLO5jnoh5fLpVvG9eD/ah+b76CfpmmQZdn3JEl+m5nPNoqCah+beQ76m81mvF6vB+fzuVPmxeGfJsRmZsUH/f1+z4AvVqnYsizrmlnonCuyjNnrhiodirgZm3LwjuO4m6bpj/l83ul0Og9vh8xe11XtUMTV2NSD9+l0eguCYLBYLJLZbLZ/dB2z102VOhRx6872ksF7OBwyez1ZVQ5F3J3ZGLzxLLxBgAyxQYbYIENskCE2yFTqDUKdeL7tMGvxcXJi8+D7tsOs3cfJic2D79uOth8nJ7YvKPq246/W/unNAwJkuLOJtfnBgtiE2v5gQWxCbX+wILYXqMODRRnbPbHhk7K2e2LDJ2Vt98SGq5693fM/G2SIDTLEBhligwyxQYbYIENskCE2yBAbZIgNMsQGGWKDDLFBhtggQ2yQITbIEBtkiA0yHAuviSZ8I4LYaqAp34ggthpoyjciiK1G6v6NCB4QIENskCE2yBAbZIgNMsQGGWKDDLFBhtggQ2yQITbIEBtkiA0yxAYZYoMMsUHm7uHJKIqCIj94PB4DM7M4joPtdjto2ro6Xat63b1W/gB+uLzYNGXziQAAAABJRU5ErkJggg=="">",0(0.0%)
4,Año[int64],Mean (sd) : 2018.2 (1.7)min < med < max:2016.0 < 2018.0 < 2021.0IQR (CV) : 3.0 (1198.6),6 distinct values,"<img src = ""data:image/png;ba

## Tratamiento de los datos del dataset `homicidios- víctimas`

In [41]:
homicidiosVictimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


### Transformación de los datos

Conversión del tipo de dato

In [43]:
#Cambio la columna a tipo datetime
homicidiosVictimas["FECHA_FALLECIMIENTO"] = pd.to_datetime(homicidiosVictimas["FECHA_FALLECIMIENTO"], format='%Y-%m-%d', errors='coerce')

In [44]:
#Una vez que se paso el tipo de dato a datetime que es una fecha y hora completa se utiliza este codigo para que solo se quede en formato fecha sin hora
homicidiosVictimas["FECHA_FALLECIMIENTO"] = homicidiosVictimas["FECHA_FALLECIMIENTO"].dt.date

#Luego se reemplazan las fechas en cero con espacios vacios a fin de su manipulación
homicidiosVictimas["FECHA_FALLECIMIENTO"] = homicidiosVictimas ["FECHA_FALLECIMIENTO"].replace(pd.NaT,"")

In [45]:
#Cambio la columna edad a numerico y luego a tipo entero
homicidiosVictimas["EDAD"] = pd.to_numeric(homicidiosVictimas["EDAD"], errors="coerce")
homicidiosVictimas["EDAD"] = homicidiosVictimas["EDAD"].astype("Int64")

In [46]:
#Utilizo Numpy para reemplazar por NaN los sin datos o vacios y luego paso a float para que cuando quiera aplicar estadisticas no tenga errores debido a esos datos
homicidiosVictimas["EDAD"] = homicidiosVictimas["EDAD"].replace([" ","SD"], np.nan)

In [47]:
homicidiosVictimas["EDAD"] = homicidiosVictimas["EDAD"].fillna(0.0)

In [48]:
#Controlo los datos de la columna
homicidiosVictimas ["EDAD"].unique()

<IntegerArray>
[19, 70, 30, 18, 29, 22, 16, 59, 65, 34, 41, 50, 38, 21, 52, 36, 20, 54,  0,
 56, 24, 78, 79, 26, 57, 37, 58, 23, 60, 42, 53, 51, 40, 87, 76, 75, 35, 80,
 43, 45, 67, 27, 55, 49, 81, 25, 33, 46, 83, 39, 28,  7, 48,  4, 82, 32, 17,
 47, 61, 10, 95, 73, 84, 66, 85,  1, 15, 13, 77, 44, 31, 62, 74, 71, 11, 86,
 69, 72, 12, 63, 92, 68, 91, 64,  5, 88]
Length: 86, dtype: Int64

In [49]:
homicidiosVictimas.head(3)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03


In [50]:
homicidiosVictimas.isnull().sum()

ID_hecho               0
FECHA                  0
AAAA                   0
MM                     0
DD                     0
ROL                    0
VICTIMA                0
SEXO                   0
EDAD                   0
FECHA_FALLECIMIENTO    0
dtype: int64

No se encuentran datos nulos, ni campos vacios

Se crea la columna `Rango etario` basada en la columna `Edad`, para mejorar el análisis de los datos 

In [51]:

# Se utiliza la función `pd.cut()`, que permite agrupar valores numéricos en intervalos o categorías


# Definir los límites de los rangos etarios y las etiquetas
bins = [0, 20, 40, 60, 80, 100]
labels = ['Menos de 20', ' Entre 21- 40', 'Entre 41-60', 'Entre 61-80', 'Más de 80']

# Utilizar pd.cut() para crear la columna de rango etario
homicidiosVictimas['Rango_etario'] = pd.cut(homicidiosVictimas['EDAD'], bins=bins, labels=labels, right=False)

homicidiosVictimas.head(3)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,Rango_etario
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01,Menos de 20
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02,Entre 61-80
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03,Entre 21- 40


A continuación se estandarizan las mayúsculas y minúsculas del dataset

In [52]:
# Se coloca la primera en mayúscula
homicidiosVictimas.columns = [x.capitalize() for x in homicidiosVictimas.columns]
# Se reemplazan los guiones por espacios
homicidiosVictimas.columns = homicidiosVictimas.columns.str.replace('_', ' ')
# Se renombran algunas columnas
homicidiosVictimas = homicidiosVictimas.rename(columns={'Id hecho': 'Id',
                                                      'Aaaa':'Año',
                                                      'Mm':'Mes',
                                                      'Dd':'Día',
                                                      'Victima':'Víctima'})
homicidiosVictimas.columns

Index(['Id', 'Fecha', 'Año', 'Mes', 'Día', 'Rol', 'Víctima', 'Sexo', 'Edad',
       'Fecha fallecimiento', 'Rango etario'],
      dtype='object')

A partir de los datos, se analiza eliminar columnas que se repiten en ambos dataset para realizar luego la unión y trabajar con un conjunto de datos.
Se proceden a eliminar del dataset `homicidios_victimas` las columnas `Fecha`, `Año`, `Mes`, `Día` y `Victima` que contienen la misma información que se encuentra en `homicidios_hechos`. 

In [53]:
#Se busca un registro para comparar con el mismo registro en el otro dataset
homicidiosHechos[homicidiosHechos["Id"]=="2019-0012"]

,Id,Num víctimas,Fecha,Año,Mes,Día,Hora completa,Hora,Lugar del hecho,Tipo de calle,...,Comuna,Xy (caba),Pos x,Pos y,Participantes,Victima,Acusado,Coordenada x,Coordenada y,Día semana
429,2019-0012,1,2019-02-17,2019,2,17,03:16:00,3,"CASTAÑARES AV. Y GORDILLO, TIMOTEO",AVENIDA,...,Comuna Comuna 8,Point (98072.98959434 94742.80804806),-58.484327,-34.676658,MOTO-AUTO,MOTO,AUTO,98072.989594,94742.808048,Domingo


In [54]:
homicidiosVictimas[homicidiosVictimas["Id"]=="2019-0012"]

,Id,Fecha,Año,Mes,Día,Rol,Víctima,Sexo,Edad,Fecha fallecimiento,Rango etario
447,2019-0012,2019-02-17,2019,2,17,CONDUCTOR,MOTO,MASCULINO,31,2019-02-20,Entre 21- 40


In [55]:
dfSummary(homicidiosVictimas)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Id[object],1. 2017-00352. 2017-00503. 2016-00414. 2018-00155. 2020-00636. 2017-01267. 2019-00108. 2017-00269. 2017-011210. 2018-009811. other,3 (0.4%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)696 (97.1%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADm0lEQVR4nO3cQWojZxRG0VdGOO1JCSHwEpI9eBG92F6EFpE9GOMaxB1PlEECMenOJNC3QDlnZDT6BxeeDfq8XK/XgcLd3g/g/0NsZMRGRmxkxEZGbGTERkZsZMRGZpmZh5mZ6/X6tvNbuHGH8/n8eWZmWZYvguNHulvX9bSu62lm7vd+DLfN72xkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERuawbdvLXz+/7/oSbp4NApnFv8yi4nc2MnfLsvyyLMunvR/C7bubmZ9n5qe9H8Ltc0bJiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MjczcyvM/P73g/h9tkgkHFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiMjNjLWVWSsq8g4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryFhXkXFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryFhXkXFGyYiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyIiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVknFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlMwyMw8zc7/3Q7g579fr9e3jB4fz+fx5XdfTXi/iNm3b9rIsy5ePwR3WdT09PT29HY/Hr3s+jtvx+vr66XK5nJ6fn+9n5u/YZmaOx+PXx8fH33Z7Hbfo4Z8f+AOBjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNzGHmz6+E7P0Qbse/9XTYtu3lcrmc5jtfCYH/atu2l5l5//iZr4Xzo3zztXDrKjL+QCAjNjJiIyM2MmIjIzYyYiMjNjJiI/MHTYp+Vs//7zEAAAAASUVORK5CYII="">",0(0.0%)
2,Fecha[datetime64[ns]],"Min: 2016-01-01Max: 2021-12-30Duration: 2,190 days",598 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABNCAYAAACxBha+AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACqElEQVR4nO3dTW7iMBiA4c9DgfCjRAgJdlxgpFmwzCHmsHOELjgBN2CHhFAQaSNCyGymqw4FG/LVSd5n78qLV7GdGsWUZSmAhh/fPQG0B7FBzUsVf9QYMxCRnsPQU1mW78+eD/zw9NiMMYPpdPo7DMOJ7djD4bA3xvwhuGaq4snWC8NwEsfxexRF2b2DkiQJVqvVZLfb9USE2BqokmVURCSKomw2m71ZDhtUMpkrHljuRVjyrVUWm+8eWe5FWPJdtDY2cVzuRVjyXbU5NhFxXu5FlJf8JmhEbI57r/ByuXSrmA/+r/axue698jwPiqL4mWXZq4i4PNlgyavYiqLoikhojLEZFo7H41kcxwebvddms5ms1+vh+XzuWE8UTryJLU3Tbp7nvxaLRafT6dwdzccTqt/vv9rsvfb7PXsuZd7EdjqdXoIgGC6Xy2w+n+/vHccTqj68ie3DaDSyOh3yhKoP72LDc/l0KeLL2HilUG++XYq4GhuvFBrBq0sRXz3ZnCbKht0/vlyKuLlns50oG3Zcw7VwqCE2qCE2qCE2qCE2qOE/CI4cb6iItPi3C8TmwPWGiki7f7tAbA5cb6i0/bcLxPYA2xsq/7T2pTcHBKghNqhhGVXW5lMssSlq+ymW2BS1/RRLbN+gDqfYKpZ7YsMnVS33xIZPqlruiQ1XPXu55z0b1BAb1BAb1BAb1BAb1BAb1BAb1BAb1BAb1BAb1BAb1BAb1BAb1BAb1BAb1BAb1BAb1HBTtyZcv+vl02cCiK0GHv2uly+fCSC2GmjKd72IrUbq/l0vDghQQ2xQQ2xQQ2xQQ2xQQ2xQQ2xQQ2xQQ2xQQ2xQQ2xQQ2xQQ2xQQ2xQQ2xQQ2xQc/PyZJIkgc0fPB6PgYhImqbBdrsdNm1cneaqPe5WK38Bgqu04fQVwa0AAAAASUVORK5CYII="">",0(0.0%)
3,Año[int64],Mean (sd) : 2018.2 (1.7)min < med < max:2016.0 < 2018.0 < 2021.0IQR (CV) : 2.0 (1206.6),6 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACH0lEQVR4nO3cT2rCQBiH4W9aK9FFggheojfwED1sj9CFJ/AG7gSREa2hGtOFVCrYqvlD5mfeZz9mhJchTia6PM8NCN1T0xMAbtFpegKhcs71zKxbcPhXnufbKufTdoR6gXOuNxwO3+I4HhQZv1qtls65d2KtztVQW7qydOM4HozH422SJOk9A7330WQyGSwWi66ZKX73IP0battXliRJ0tFo9FlgaK/yybTctRW10ZWlpas5LrjpHrWJlaXtqznOhfxjivtEnIQcqplxn4gjNvwhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhIfhDKW1T8gyu2YOewyXUgJQ9g2v2uOdwCTUshc/gmj32OVxCDVCJM7hmD3oOl1BRmTrfcSNUVKLud9wIFVWp9R03QkWl6nrHjQ1/SCBUSCBUSOAeFSclt5fiw+HwUuV8fiNUmFn57aXdbhdlWfaapumHmRV9WPEnQsWPUo9vZ7PZYDqd9vf7/XMdkyNUnCm6vbRcLmt9dMuPKUggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggV

In [56]:
# Se eliminan las columnas repetidas
homicidiosVictimas = homicidiosVictimas.drop(['Fecha', 'Año', 'Mes', 'Día', 'Víctima'], axis=1)
homicidiosVictimas.columns

Index(['Id', 'Rol', 'Sexo', 'Edad', 'Fecha fallecimiento', 'Rango etario'], dtype='object')

## Unificación de los dataset

Por medio de la función merge se unifican los dataset transformados anteriormente

In [57]:
siniestros= homicidiosVictimas.merge(homicidiosHechos, on="Id", how="left")
siniestros.columns

Index(['Id', 'Rol', 'Sexo', 'Edad', 'Fecha fallecimiento', 'Rango etario',
       'Num víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora completa', 'Hora',
       'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Pos x', 'Pos y',
       'Participantes', 'Victima', 'Acusado', 'Coordenada x', 'Coordenada y',
       'Día semana'],
      dtype='object')

In [58]:
siniestros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Id                     717 non-null    object        
 1   Rol                    717 non-null    object        
 2   Sexo                   717 non-null    object        
 3   Edad                   717 non-null    Int64         
 4   Fecha fallecimiento    717 non-null    object        
 5   Rango etario           717 non-null    category      
 6   Num víctimas           717 non-null    int64         
 7   Fecha                  717 non-null    datetime64[ns]
 8   Año                    717 non-null    int64         
 9   Mes                    717 non-null    int64         
 10  Día                    717 non-null    int64         
 11  Hora completa          717 non-null    object        
 12  Hora                   716 non-null    Int64         
 13  Lugar

In [59]:
siniestros["Num víctimas"].unique()

array([1, 2, 3], dtype=int64)

In [60]:
siniestros ["Num víctimas"] = siniestros ["Num víctimas"].fillna(0.0)
siniestros ["Num víctimas"] = siniestros ["Num víctimas"].astype ("Int64")

In [61]:
siniestros["Num víctimas"].unique()

<IntegerArray>
[1, 2, 3]
Length: 3, dtype: Int64

In [62]:
siniestros["Año"].unique()

array([2016, 2017, 2018, 2019, 2020, 2021], dtype=int64)

In [63]:
siniestros ["Año"]= siniestros ["Año"].astype ("Int64")
siniestros["Año"].unique()

<IntegerArray>
[2016, 2017, 2018, 2019, 2020, 2021]
Length: 6, dtype: Int64

In [64]:
siniestros ["Mes"]= siniestros ["Mes"].astype ("Int64")
siniestros["Mes"].unique()

<IntegerArray>
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Length: 12, dtype: Int64

In [65]:
siniestros ["Día"]= siniestros ["Día"].astype ("Int64")
siniestros["Día"].unique()

<IntegerArray>
[ 1,  2,  3, 10, 21, 24, 29,  8, 14, 15, 17, 28,  4, 12, 13, 19, 23, 30, 31,
 11, 20, 22, 25, 26, 16, 18, 27,  7,  9,  5,  6]
Length: 31, dtype: Int64

In [66]:
siniestros.head(3)

,Id,Rol,Sexo,Edad,Fecha fallecimiento,Rango etario,Num víctimas,Fecha,Año,Mes,...,Comuna,Xy (caba),Pos x,Pos y,Participantes,Victima,Acusado,Coordenada x,Coordenada y,Día semana
0,2016-0001,CONDUCTOR,MASCULINO,19,2016-01-01,Menos de 20,1,2016-01-01,2016,1,...,Comuna Comuna 8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378,Viernes
1,2016-0002,CONDUCTOR,MASCULINO,70,2016-01-02,Entre 61-80,1,2016-01-02,2016,1,...,Comuna Comuna 9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420,Sabado
2,2016-0003,CONDUCTOR,MASCULINO,30,2016-01-03,Entre 21- 40,1,2016-01-03,2016,1,...,Comuna Comuna 1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,106684.290900,99706.576878,Domingo


In [68]:
siniestros.describe()

,Edad,Num víctimas,Fecha,Año,Mes,Día,Hora,Pos x,Pos y,Coordenada x,Coordenada y
count,717.0,717.0,717,717.0,717.0,717.0,716.0,704.000000,704.000000,702.000000,702.000000
mean,39.050209,1.061367,2018-09-09 01:24:21.087866112,2018.174338,6.688982,15.953975,11.554469,-58.441664,-34.619700,101980.113004,101070.596233
min,0.0,1.0,2016-01-01 00:00:00,2016.0,1.0,1.0,0.0,-58.529942,-34.705250,93889.867899,91571.233073
25%,24.0,1.0,2017-04-01 00:00:00,2017.0,4.0,9.0,6.0,-58.476556,-34.643918,98744.378735,98387.965943
50%,35.0,1.0,2018-07-07 00:00:00,2018.0,7.0,16.0,11.0,-58.444513,-34.622866,101721.590022,100711.696729
75%,55.0,1.0,2019-12-18 00:00:00,2019.0,10.0,23.0,17.0,-58.401841,-34.597013,105630.884244,103582.742374
max,95.0,3.0,2021-12-30 00:00:00,2021.0,12.0,31.0,23.0,-58.356082,-34.534654,109831.098614,110496.053085
std,22.010669,0.257024,NaN,1.672655,3.57514,8.674876,6.700055,0.046397,0.035292,4253.717694,3914.938863


In [69]:
dfSummary(siniestros)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Id[object],1. 2017-00352. 2017-00503. 2016-00414. 2018-00155. 2020-00636. 2017-01267. 2019-00108. 2017-00269. 2017-011210. 2018-009811. other,3 (0.4%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)696 (97.1%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADm0lEQVR4nO3cQWojZxRG0VdGOO1JCSHwEpI9eBG92F6EFpE9GOMaxB1PlEECMenOJNC3QDlnZDT6BxeeDfq8XK/XgcLd3g/g/0NsZMRGRmxkxEZGbGTERkZsZMRGZpmZh5mZ6/X6tvNbuHGH8/n8eWZmWZYvguNHulvX9bSu62lm7vd+DLfN72xkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERuawbdvLXz+/7/oSbp4NApnFv8yi4nc2MnfLsvyyLMunvR/C7bubmZ9n5qe9H8Ltc0bJiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MjczcyvM/P73g/h9tkgkHFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiMjNjLWVWSsq8g4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryFhXkXFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryFhXkXFGyYiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyIiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVknFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlMwyMw8zc7/3Q7g579fr9e3jB4fz+fx5XdfTXi/iNm3b9rIsy5ePwR3WdT09PT29HY/Hr3s+jtvx+vr66XK5nJ6fn+9n5u/YZmaOx+PXx8fH33Z7Hbfo4Z8f+AOBjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNzGHmz6+E7P0Qbse/9XTYtu3lcrmc5jtfCYH/atu2l5l5//iZr4Xzo3zztXDrKjL+QCAjNjJiIyM2MmIjIzYyYiMjNjJiI/MHTYp+Vs//7zEAAAAASUVORK5CYII="">",0(0.0%)
2,Rol[object],1. CONDUCTOR2. PEATON3. PASAJERO_ACOMPAÑANTE4. CICLISTA5. SD,330 (46.0%)267 (37.2%)80 (11.2%)29 (4.0%)11 (1.5%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABzCAYAAACPdnBjAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACaklEQVR4nO3dQWrbUBiF0d/GpPVExgSyjCwgi+hiuwgvIPsIJhrUaSbuoKUUmriGPN/nSues4MV8SEjoksXxeCxIWPY+APMhNmLERsyiqtZVddP7IFfq9Xg8HnofYipWt7e3X4Zh2PY+yDUax3G/WCy+Cq6N1TAM24eHh8Nms3npfZhr8vz8/Hm3222fnp5uqkpsDayqqjabzcvd3d233oe5QuveB5gSDwjEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERsyq6ufnNL0Pcm38Ju2txnHc73a7bfmc5i/jOO6r6rX3OabCZ+Gn+Sy8oYXdKCkeEIiZ023ULbGz2ayrLKX6m8W6ylLqOsxpXeXVTmceEIgRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzFiI0ZsxIiNmFmsq6b+9/0vZrOuspTqz2fhxFhXEeMBgZhWt1G3KP6pybrKcolzfHhdZbnEuVqtqyb92oQ2PCAQIzZixEaM2IgRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzEfXldZLnGuJusqyyXO4bNwYqyriDl1ZXO1oql3By9GLLT25uDFiIVLODV4MWKhKS91iREbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsRHz5rrKYopLeHddZTFFazYIxFhXEfPnlc2VjIv6va6ypuLSlsMwbO/v7+vXnG8O/3eUTpZVVev1+nvvgzB9XuoSIzZixEaM2IgRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzFiI2ZZVXU4HD71PgjTtxzHcf/4+GhNxcXZIBBjXUWMBwRixEbMDxqt/w3kjlieAAAAAElFTkSuQmCC"">",0(0.0%)
3,Sexo[object],1. MASCULINO2. FEMENINO3. SD,545 (76.0%)166 (23.2%)6 (0.8%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABFCAYAAABdVZTTAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABnklEQVR4nO3bQWrCUBSF4RMJUicvhICLcREutot4uwnBN6jiJB3UgnQoL+dd7P+t4IJ/1RRPt66rAIdd6wPwf3SSDpL2rQ9Bdfd1Xa+tj3jWT9N0TimNrQ9BXaWUpeu6z0jB9Sml8XQ6XYdhuLU+BnVcLpePnPM4z/NeUpzYJGkYhtvxePxqfQyqOrQ+4C8eEGBDbLAhNtgQG2yIDTbEBhtigw2xwYbYYENssCE22BAbbIgNNsQGm176+f1T60NQT9TXsy+lLDnnUQF//4TXlVIWSffWdzxjg/C+wm0QOqZ8cIn4zhbuLxJ1hFtXRVwFoY5Q66qoqyDUEXFdxVPxm+KfurAhNtgQG2yIDTbEBhtigw2xwYbYYENssCE22BAbbIgNNsQGG2KDTah1VZQ7sI1w66qIqyDUwQYBNqyrYMMDAmx+P0bFRxe21k/TdJYk5nPY2i6lND52o5EeEvCG+M4GG2KDDbHBhthgQ2ywITbYEBtsiA02xAYbYoMNscGG2GBDbLAhNtgQG2yIDTb9YzonMZ/DxtggwIYpH2y+ARDSg2w2upFrAAAAAElFTkSuQmCC"">",0(0.0%)
4,Edad[Int64],Mean (sd) : 39.1 (22.0)min < med < max:0.0 < 35.0 < 95.0IQR (CV) : 31.0 (1.8),86 distinct values,"<img src = ""data:image/png;base64, iVBORw0

In [78]:
#Creo una lista con el orden 
nuevoOrden=["Id", "Fecha", "Año", "Mes", "Día","Día semana", "Hora completa", "Hora", "Num víctimas", "Participantes", "Rol", "Acusado", "Victima", "Sexo", "Edad", "Rango etario", "Fecha fallecimiento", "Lugar del hecho", "Tipo de calle", "Calle", "Cruce", "Dirección normalizada", "Comuna", "Xy (caba)", "Coordenada x", "Coordenada y", "Pos x", "Pos y"]

#Reorganizo el dataframe untilizando la lista y asignandola nuevamente aldataframe siniestros
siniestrosLimpio = siniestros[nuevoOrden]
siniestrosLimpio.columns


Index(['Id', 'Fecha', 'Año', 'Mes', 'Día', 'Día semana', 'Hora completa',
       'Hora', 'Num víctimas', 'Participantes', 'Rol', 'Acusado', 'Victima',
       'Sexo', 'Edad', 'Rango etario', 'Fecha fallecimiento',
       'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Coordenada x',
       'Coordenada y', 'Pos x', 'Pos y'],
      dtype='object')

In [79]:
siniestrosLimpio.head(3)

,Id,Fecha,Año,Mes,Día,Día semana,Hora completa,Hora,Num víctimas,Participantes,...,Tipo de calle,Calle,Cruce,Dirección normalizada,Comuna,Xy (caba),Coordenada x,Coordenada y,Pos x,Pos y
0,2016-0001,2016-01-01,2016,1,1,Viernes,04:00:00,4,1,MOTO-AUTO,...,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",Comuna Comuna 8,Point (98896.78238426 93532.43437792),98896.782384,93532.434378,-58.475340,-34.687570
1,2016-0002,2016-01-02,2016,1,2,Sabado,01:15:00,1,1,AUTO-PASAJEROS,...,GRAL PAZ,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",Comuna Comuna 9,Point (95832.05571093 95505.41641999),95832.055711,95505.416420,-58.508775,-34.669777
2,2016-0003,2016-01-03,2016,1,3,Domingo,07:00:00,7,1,MOTO-AUTO,...,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,Comuna Comuna 1,Point (106684.29090040 99706.57687843),106684.290900,99706.576878,-58.390403,-34.631894


In [81]:
siniestrosLimpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Id                     717 non-null    object        
 1   Fecha                  717 non-null    datetime64[ns]
 2   Año                    717 non-null    Int64         
 3   Mes                    717 non-null    Int64         
 4   Día                    717 non-null    Int64         
 5   Día semana             717 non-null    object        
 6   Hora completa          717 non-null    object        
 7   Hora                   716 non-null    Int64         
 8   Num víctimas           717 non-null    Int64         
 9   Participantes          717 non-null    object        
 10  Rol                    717 non-null    object        
 11  Acusado                717 non-null    object        
 12  Victima                717 non-null    object        
 13  Sexo 

In [82]:
siniestrosLimpio["Fecha"]=siniestrosLimpio["Fecha"].dt.date

In [83]:
siniestrosLimpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Id                     717 non-null    object  
 1   Fecha                  717 non-null    object  
 2   Año                    717 non-null    Int64   
 3   Mes                    717 non-null    Int64   
 4   Día                    717 non-null    Int64   
 5   Día semana             717 non-null    object  
 6   Hora completa          717 non-null    object  
 7   Hora                   716 non-null    Int64   
 8   Num víctimas           717 non-null    Int64   
 9   Participantes          717 non-null    object  
 10  Rol                    717 non-null    object  
 11  Acusado                717 non-null    object  
 12  Victima                717 non-null    object  
 13  Sexo                   717 non-null    object  
 14  Edad                   717 non-null    Int

In [84]:
dfSummary(siniestrosLimpio, is_collapsible=True)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Id[object],1. 2017-00352. 2017-00503. 2016-00414. 2018-00155. 2020-00636. 2017-01267. 2019-00108. 2017-00269. 2017-011210. 2018-009811. other,3 (0.4%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)696 (97.1%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADm0lEQVR4nO3cQWojZxRG0VdGOO1JCSHwEpI9eBG92F6EFpE9GOMaxB1PlEECMenOJNC3QDlnZDT6BxeeDfq8XK/XgcLd3g/g/0NsZMRGRmxkxEZGbGTERkZsZMRGZpmZh5mZ6/X6tvNbuHGH8/n8eWZmWZYvguNHulvX9bSu62lm7vd+DLfN72xkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERuawbdvLXz+/7/oSbp4NApnFv8yi4nc2MnfLsvyyLMunvR/C7bubmZ9n5qe9H8Ltc0bJiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MjczcyvM/P73g/h9tkgkHFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiMjNjLWVWSsq8g4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryFhXkXFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryFhXkXFGyYiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyIiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVknFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlMwyMw8zc7/3Q7g579fr9e3jB4fz+fx5XdfTXi/iNm3b9rIsy5ePwR3WdT09PT29HY/Hr3s+jtvx+vr66XK5nJ6fn+9n5u/YZmaOx+PXx8fH33Z7Hbfo4Z8f+AOBjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNzGHmz6+E7P0Qbse/9XTYtu3lcrmc5jtfCYH/atu2l5l5//iZr4Xzo3zztXDrKjL+QCAjNjJiIyM2MmIjIzYyYiMjNjJiI/MHTYp+Vs//7zEAAAAASUVORK5CYII="">",0(0.0%)
2,Fecha[object],1. 2017-02-262. 2016-02-283. 2017-11-194. 2017-12-115. 2018-04-276. 2016-06-137. 2018-08-038. 2020-12-259. 2018-12-2210. 2018-12-1811. other,3 (0.4%)3 (0.4%)3 (0.4%)3 (0.4%)3 (0.4%)3 (0.4%)3 (0.4%)3 (0.4%)3 (0.4%)3 (0.4%)687 (95.8%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADfUlEQVR4nO3cMWobUQBF0S8jTNyMEAItRovwYrMI7UYITREbN0oRpwhRAgH7DijnVCHVFJfR5PNfVtfrdUDhYekH4P8hNjJiIyM2MmIjIzYyYiMjNjJiI7MaYzyNMcb1en1Z+Fm4c+vdbvc8xhir1eqr4PhMD9M0badp2o4xHpd+GO6bbzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiI7Oe5/n8/ue3RZ+Eu2eDQGblv8yi4s1GxrqKjHUVGUcfZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkDF7IuBZOxuCFjDcbGYMXMgYvZBx9kBEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRMXgh41o4GYMXMr7ZyPgZJWNdRca6ioxvNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MtZVZFwLJ2NdRcabjYzBCxmDFzKOPsiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyBi8kHEtnIzBCxlvNjIGL2QMXsg4+iAjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MtZVZGwQyFhXkfFmI2NdRca6ioyjDzJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MgYvZFwLJ2PwQsabjYzBCxmDFzKOPsiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyBi8kHEtnIzBCxnfbGT8jJKxriJjXUXGNxsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxnrKjKuhZOxriLjzUbG4IWMwQsZRx9kxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGQMXsi4Fk7G4IXMzzebK+F8pLdbv5Tr3W73/L5BgA8xz/P51oBqPU3T9nA4vGw2m9elHo77cblcvhyPx+3pdHocY/wa2xhjbDab1/1+/22Rp+MePd36S+dsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZNZj/LgWsvSDcB/+1tJ6nufz8Xjcjj9cC4F/9T6i+m1A5Vo4n+HmtXCDFzL+gUBGbGTERkZsZMRGRmxkxEZGbGTERuY7CLkQJ3942gEAAAAASUVORK5CYII="">",0(0.0%)
3,Año[Int64],Mean (sd) : 2018.2 (1.7)min < med < max:2016.0 < 2018.0 < 2021.0IQR (CV) : 2.0 (1206.6),6 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACH0lEQVR4nO3cT2rCQBiH4W9aK9FFggheojfwED1sj9CFJ/AG7gSREa2hGtOFVCrYqvlD5mfeZz9mhJchTia6PM8NCN1T0xMAbtFpegKhcs71zKxbcPhXnufbKufTdoR6gXOuNxwO3+I4HhQZv1qtls65d2KtztVQW7qydOM4HozH422SJOk9A7330WQyGSwWi66ZKX73IP0battXliRJ0

In [86]:
siniestrosLimpio.to_csv("siniestosLimpio.csv", index=False)